In [1]:
import pandas as pd
import psycopg2
import numpy as np
import sys
import csv
import re
import string
from nltk import word_tokenize
from collections import Counter
from itertools import chain
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
from Database import database
import Database
from Connection import connection
from Preprocessing import preprocessing


con = connection.connect(Database.params_dic)

column_names = ["user_name", "comments_user", "video_title", "channel", "views", "posted"]

df = database.postgresql_to_dataframe(con, "select * from youtube_comments", column_names)

con.close()

df = df.drop_duplicates(['user_name', 'comments_user', 'posted'])

df['comments_clean'] = pd.DataFrame(df['comments_user'].apply(lambda x: preprocessing.clean_text(x)))

v = df['comments_clean'].str.split().tolist()
c = Counter(chain.from_iterable(v))
df['comments_clean']=[' '.join([j for j in i if c[j] > 1]) for i in v]

df['comments_tokenize'] = pd.DataFrame(df['comments_clean'].apply(lambda x: preprocessing.tokenize(x)))

df['comments_slangword'] = pd.DataFrame(df['comments_tokenize'].apply(lambda x: preprocessing.slangword(x)).apply(lambda x: preprocessing.tokenize(x)))

term_dict = {}
for document in df['comments_slangword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = preprocessing.stemmed_wrapper(term)
    
def get_stemmed_term(document):
        return [term_dict[term] for term in document]
    
df['comments_stemmed'] = df['comments_slangword'].swifter.apply(get_stemmed_term)

df['comments_stopwords'] = pd.DataFrame(df['comments_stemmed'].apply(lambda x : preprocessing.stopwords(x)).apply(lambda x: preprocessing.tokenize(x)))

df['text_string'] = pd.DataFrame(df['comments_stemmed'].apply(lambda x : preprocessing.stopwords(x)))

df['text_string'] = df['text_string'].str.strip().replace('',np.nan)

df = df.dropna(how='any', axis=0).reset_index()

df.columns = ["document_no","user_name", "comments_user", "video_title", "channel", "views", "posted", "comments_clean",
              "comments_tokenize", "comments_slangword", "comments_stemmed", "comments_stopwords", "text_string"]

#df = df.reset_index()

#df.columns = ["document_no","user_name", "comments_user", "video_title", "channel", "views", "posted", "comments_clean",
              #"comments_tokenize", "comments_slangword", "comments_stemmed", "comments_stopwords", "text_string"]

df.to_csv(Database.path_data + 'youtube_comments_preprocessing.csv', encoding="UTF-8", sep=',', index=False)

database.createTableDatabasePreprocessing()
database.intoDatabasePreprocessing()

C:\Users\Zukkiii_\Anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Connecting to the PostgreSQL database...
Connection successful



Connecting to the PostgreSQL database...
Connection successful
Table created successfully
Connecting to the PostgreSQL database...
Connection successful


In [ ]:
df['text_string'] = df['text_string'].str.strip().replace('',np.nan)

df = df.dropna(how='any', axis=0).reset_index()

df.columns = ["document_no","user_name", "comments_user", "video_title", "channel", "views", "posted", "comments_clean",
              "comments_tokenize", "comments_slangword", "comments_stemmed", "comments_stopwords", "text_string"]

#df = df.reset_index()

#df.columns = ["document_no","user_name", "comments_user", "video_title", "channel", "views", "posted", "comments_clean",
              #"comments_tokenize", "comments_slangword", "comments_stemmed", "comments_stopwords", "text_string"]

df.to_csv(Database.path_data + 'youtube_comments_preprocessing.csv', encoding="UTF-8", sep=',', index=False)

database.createTableDatabasePreprocessing()
database.intoDatabasePreprocessing()